In [1]:
import numpy as np
import copy
import time
import os

for root,dirs,files in os.walk(r'sokoban_benchmarks'):
                               print("Loading Data...")

levels=[]
for l in range(len(files)):
    f = open(r'sokoban_benchmarks/%s'%files[l])
    line = f.readline()
    data_list = []
    while line:
        num = list(map(int,line.split()))
        data_list.append(num)
        line = f.readline()
    f.close()
    data_array = np.array(data_list)

    #print(data_list)
    s_row = data_list[0][1]
    s_column = data_list[0][0]
    s_wall = data_list[1]
    s_box = data_list[2]
    s_purpose = data_list[3]
    s_person = data_list[4]
    s_person[0] -= 1
    s_person[1] -= 1

    aaa = np.ones((s_row,s_column))
    #print(aaa)

    s_wall_t = tuple(s_wall)
    wall_volume = s_wall_t[0]
    del s_wall[0]
    for i in range(len(s_wall)):
        s_wall[i] -= 1
#print(wall_volume)
#print(s_wall)

    s_box_t = tuple(s_box)
    box_volume = s_box_t[0]
    del s_box[0]
    for j in range(len(s_box)):
        s_box[j] -= 1
#print(box_volume)
#print(s_box)

    s_purpose_t = tuple(s_purpose)
    purpose_volume = s_purpose_t[0]
    del s_purpose[0]
    for k in range(len(s_purpose)):
        s_purpose[k] -= 1
#print(purpose_volume)
#print(s_purpose)

    for i in range(wall_volume):
        rr = s_wall[2*i]
        cc = s_wall[2*i+1]
        aaa[rr][cc] = 0

    for i in range(box_volume):
        rr = s_box[2*i]
        cc = s_box[2*i+1]
        aaa[rr][cc] = 3

    for i in range(purpose_volume):
        rr = s_purpose[2*i]
        cc = s_purpose[2*i+1]
        aaa[rr][cc] = 4
    
    aaa[s_person[0],s_person[1]] = 2
    aaa=aaa.astype(np.int8)
    levels.append(aaa)
print(len(levels))

Loading Data...
15


In [2]:
def person_position(P_1):
    
    # search the position of player
    for i in range(len(P_1)): #length
        for j in range(len(P_1[0])): #width
            if P_1[i][j] == 2 or P_1[i][j] == 6:
                return ([i,j])

def Trans_array_str(P):
    a = ''
    for i in range(len(P)):
        for j in range(len(P[0])):
            a += str(P[i][j])
    return a

def match(P_1,P_end):
    ## check if current state match final state
    for i in range(len(P_1)):
        if P_1[i] !='3' and P_end[i] =='4':
            return False   
    return True 

def best_move(pathfinal):
    moves = ''
    for i in range(len(pathfinal)):
        if pathfinal[i]==[0,-1]:
            moves+='%d L '%(i+1)
        if pathfinal[i]==[-1,0]:
            moves+='%d U '%(i+1)
        if pathfinal[i]==[0,1]:
            moves+='%d R '%(i+1)
        if pathfinal[i]==[1,0]:
            moves+='%d D '%(i+1)
    return moves

def death_block(P,length_c,P_end):
    boxes=[]
    storages=[]
    for i in range(len(P)):
        if P[i]==str(3):
            boxes.append(i)
    for i in range(len(P_end)):
        if P_end[i]==str(4):
            storages.append(i)
    for i in range(len(boxes)):
        if (P[boxes[i]-1]==str(0) and P[boxes[i]-length_c]==str(0)) or (P[boxes[i]-1]==str(0) and P[boxes[i]+length_c]==str(0)) or (P[boxes[i]+1]==str(0) and P[boxes[i]-length_c]==str(0)) or (P[boxes[i]+1]==str(0) and P[boxes[i]+length_c]==str(0)):
            if boxes[i] not in storages:
                return True
    return False
    
    

In [11]:
iteration=0
def BFS(aaa):
    start =time.clock()

#P=np.array([[0,0,0,0,0,0,0],[0,1,1,1,1,2,0],[0,4,3,1,3,4,0],[0,1,1,1,1,1,0],[0,0,0,0,0,0,0]])
#P = np.array([[0,0,0,0,0,0,0,0],[0,4,1,0,1,1,1,0],[0,1,1,3,1,1,1,0],[0,1,1,1,0,1,0,0],[0,0,1,0,1,3,4,0],[0,1,1,1,3,1,1,0],[0,1,1,4,0,1,2,0],[0,0,0,0,0,0,0,0]])

    P = np.array(copy.deepcopy(aaa))
    print(P)

    explore = [[0,-1],[-1,0],[0,1],[1,0]]
    length_r = P.shape[0]
    length_c = P.shape[1]
#print(length_r)
#print(length_c)

    a = Trans_array_str(P)
#print(a)

    P_transform = a.replace('4','1')
#print(P_transform)

    P_end = a.replace('2','1')
    P_end = P_end.replace('3','1')
#print(P_end)

    states=[] ## frontier, record states to explore
    iterate_max=1000 ## death loop maximum
    person=person_position(P)

    visited = {a:0} ## record visited states

    pathfinal=[]

    states=[[P_transform,[],person]]
    while(len(states)>0):

        P = states[0][0]  ##initial status
        path = states[0][1]
        person = states[0][2] 

        if len(path)>iterate_max:                                 ##deal with too long loop
            break
        
        if match(P,P_end):   
            if len(pathfinal)==0:
                pathfinal=path
                continue                                                ##check if ok
            elif len(path)<len(pathfinal):
                pathfinal=path
                continue
            
        states=states[1:]  ##explore the current status,update the frontier
        nnn = 1
        global iteration
        iteration=iteration+1
        for i,exp in enumerate(explore):
            neg=[person[0]+exp[0],person[1]+exp[1]]##the neighbour
        #print(neg)
            nneg=[person[0]+2*exp[0],person[1]+2*exp[1]]   ##the neighbour of the neighbour
        #print(nneg)    
            #if not (neg[0]>=0 and neg[0]<len(P) and neg[1]>=0 and neg[1]<len(P[0])):
            if not ( neg[0]>0 and neg[0]<(length_r-1) and neg[1]>0 and neg[1]<(length_c-1) ):
                continue     # out of boundary
        #print('ssss')
        #print(P[neg[0]*length_c + neg[1] ])
            if P[ neg[0]*length_c + neg[1] ]==str(1):           ## only the person move
                Pnew_1 = P[0:(neg[0]*length_c + neg[1] )] + '2' + P[(neg[0]*length_c + neg[1] +1):]

                Pnew_2 = Pnew_1[0:(person[0]*length_c + person[1] ) ] + '1' + Pnew_1[(person[0]*length_c + person[1] +1):]
                #Pnew[person[0]][person[1]]=1 
            #print('Pnew_2')
            #print(Pnew_2)     
                    
                if Pnew_2 not in visited:## check if visited
                    
                    visited[Pnew_2] = 0
                
                    states.append([Pnew_2,path+[exp],neg])  ##add new status in frontier
                    
            elif P[neg[0]*length_c + neg[1] ]==str(3) and P[nneg[0]*length_c + nneg[1] ]==str(1):
                #Pnew_1 = ''
                Pnew_1 = P[0:(neg[0]*length_c + neg[1] )] + '2' + P[(neg[0]*length_c + neg[1] +1):]
                Pnew_2 = Pnew_1[0:(person[0]*length_c + person[1] ) ] + '1' + Pnew_1[(person[0]*length_c + person[1] +1):]
                Pnew_3 = Pnew_2[0:(nneg[0]*length_c + nneg[1] ) ] + '3' + Pnew_2[(nneg[0]*length_c + nneg[1] +1):]

                if Pnew_3 not in visited:                                            ## check if visited
                    visited[Pnew_3] = 0
                    if not death_block(Pnew_3,length_c,P_end):
                        states.append([Pnew_3,path+[exp],neg])  ##add new status in frontier
        #print(pathfinal)
    print(best_move(pathfinal))
    end = time.clock()
    print('Running time: %s Seconds'%(end-start))

#for i in range(len(levels)):
    #print('level %d:'%i)
BFS(levels[8])
print(iteration)

E:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 0 1 1 1 0 1 1 1 1 1 0]
 [0 0 0 0 0 1 1 1 0 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 0 3 0 3 0 0 1 0 0 0 0 0 0]
 [0 1 1 1 0 1 0 0 1 0 0 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 0 3 0 0 1 0 2 0 1 1 1 0]
 [0 1 1 0 1 1 1 1 0 0 0 4 4 4 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1 U 2 L 3 L 4 D 5 D 6 L 7 L 8 L 9 U 10 D 11 R 12 R 13 R 14 U 15 U 16 L 17 L 18 L 19 L 20 U 21 L 22 L 23 D 24 R 25 R 26 R 27 R 28 R 29 R 30 R 31 R 32 R 33 R 34 U 35 R 36 D 37 L 38 L 39 L 40 L 41 U 42 U 43 U 44 L 45 L 46 L 47 L 48 L 49 D 50 D 51 L 52 D 53 R 54 R 55 R 56 R 57 R 58 R 59 R 60 R 61 R 62 R 63 U 64 R 65 D 66 D 67 L 68 U 69 L 70 L 71 L 72 L 73 U 74 U 75 U 76 L 77 L 78 L 79 D 80 D 81 U 82 U 83 L 84 L 85 D 86 D 87 D 88 R 89 R 90 R 91 R 92 R 93 R 94 R 95 R 96 D 97 U 98 U 99 R 100 D 101 D 
Running time: 62.376487499999996 Seconds
766292


E:\Python\Anaconda\lib\site-packages\ipykernel_launcher.py:93: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [90]:
P='000000311000000'
death_block(P,5,P)


True

1
